In [ ]:
import pandas as pd
import json
import pickle
import matplotlib as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

with open('/content/gdrive/MyDrive/kaggle_dataset/News_Category_Dataset_v2.json', 'r') as f:
    mydata = pd.DataFrame([json.loads(l) for l in f.readlines()])

mydata.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [ ]:
mydata

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


In [ ]:
# Generating the summary of entries per category
categories = mydata.groupby('category')
print("total categories:", categories.ngroups)
print(categories.size())

total categories: 41
category
ARTS               1509
ARTS & CULTURE     1339
BLACK VOICES       4528
BUSINESS           5937
COLLEGE            1144
COMEDY             5175
CRIME              3405
CULTURE & ARTS     1030
DIVORCE            3426
EDUCATION          1004
ENTERTAINMENT     16058
ENVIRONMENT        1323
FIFTY              1401
FOOD & DRINK       6226
GOOD NEWS          1398
GREEN              2622
HEALTHY LIVING     6694
HOME & LIVING      4195
IMPACT             3459
LATINO VOICES      1129
MEDIA              2815
MONEY              1707
PARENTING          8677
PARENTS            3955
POLITICS          32739
QUEER VOICES       6314
RELIGION           2556
SCIENCE            2178
SPORTS             4884
STYLE              2254
STYLE & BEAUTY     9649
TASTE              2096
TECH               2082
THE WORLDPOST      3664
TRAVEL             9887
WEDDINGS           3651
WEIRD NEWS         2670
WELLNESS          17827
WOMEN              3490
WORLD NEWS         2177
WORLDPOST 

In [ ]:
def category_cleaner(x):
    
   if x == "SCIENCE" or x=="TECH":
        return "SCIENCE AND TECHNOLOGY"
   else:
        return x

mydata['category'] = mydata.category.apply(category_cleaner)

In [ ]:
# Initializing data frame
new_data =pd.DataFrame()
# Selecting N samples for training 
Nsamples = 3405
for cat in ["CRIME","ENTERTAINMENT","SPORTS","SCIENCE AND TECHNOLOGY","TRAVEL","BUSINESS","POLITICS"]:
  print(cat)
  new_data = new_data.append(mydata[mydata['category']==cat].sample(n=Nsamples)) 

CRIME
ENTERTAINMENT
SPORTS
SCIENCE AND TECHNOLOGY
TRAVEL
BUSINESS
POLITICS


In [ ]:
# Combining two categories
new_data["text"] = new_data.headline	+""+ new_data.short_description
new_data.head()

,category,headline,authors,link,short_description,date,text
78764,CRIME,South Dakota High School Principal Injured In ...,"Dirk Lammers, AP",https://www.huffingtonpost.com/entry/south-dak...,"The suspect, a student, is in custody.",2015-09-30,South Dakota High School Principal Injured In ...
29392,CRIME,Terrifying Video Shows 2 Bullets Narrowly Miss...,David Moye,https://www.huffingtonpost.com/entry/arizona-b...,A man upset with a neighboring business was ch...,2017-04-13,Terrifying Video Shows 2 Bullets Narrowly Miss...
102669,CRIME,The Real Prejudice Behind the Garner/Brown Dec...,"Amy Dardashtian, ContributorTelevision Corresp...",https://www.huffingtonpost.com/entry/the-real-...,While media coverage has focused on the growin...,2014-12-30,The Real Prejudice Behind the Garner/Brown Dec...
82921,CRIME,Elementary School Teacher Struck And Killed By...,,https://www.huffingtonpost.com/entry/roller-co...,He jumped the fence to retrieve his dropped ph...,2015-08-14,Elementary School Teacher Struck And Killed By...
38365,CRIME,Videos Capture Moment Gunman Opens Fire At Ist...,Mary Papenfuss,https://www.huffingtonpost.com/entry/cctv-capt...,Dozens of victims at the popular hot spot were...,2017-01-02,Videos Capture Moment Gunman Opens Fire At Ist...


In [ ]:
new_data.head()
categories = new_data.groupby('category')
print("total categories:", categories.ngroups)
print(categories.size())

total categories: 7
category
BUSINESS                  3405
CRIME                     3405
ENTERTAINMENT             3405
POLITICS                  3405
SCIENCE AND TECHNOLOGY    3405
SPORTS                    3405
TRAVEL                    3405
dtype: int64


In [ ]:
del new_data["short_description"]
del new_data["headline"]
del new_data["link"]
del new_data["date"]
new_data.head()

,category,authors,text
78764,CRIME,"Dirk Lammers, AP",South Dakota High School Principal Injured In ...
29392,CRIME,David Moye,Terrifying Video Shows 2 Bullets Narrowly Miss...
102669,CRIME,"Amy Dardashtian, ContributorTelevision Corresp...",The Real Prejudice Behind the Garner/Brown Dec...
82921,CRIME,,Elementary School Teacher Struck And Killed By...
38365,CRIME,Mary Papenfuss,Videos Capture Moment Gunman Opens Fire At Ist...


In [ ]:
filter_data=new_data

In [ ]:
X_message=filter_data.iloc[:,2]
Y_class=filter_data.iloc[:,0]

##Extract Features

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
my_cv=CountVectorizer()
my_tfidf=TfidfVectorizer(stop_words='english')

In [ ]:
XX=my_tfidf.fit_transform(X_message)

In [ ]:
f_n=my_tfidf.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(XX,Y_class,test_size=.001,random_state=101)

#Models


##Logistic Regression

In [ ]:
cl=LogisticRegression(solver='lbfgs')
lr=cl.fit(X_train,Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


###Accuracy

In [ ]:
YA=Y_test
Yp=lr.predict(X_test)
from sklearn.metrics import accuracy_score
acc=accuracy_score(YA,Yp)
acc

0.875

##Random Forest

In [ ]:
lr = RandomForestClassifier(n_estimators=100 ,criterion='entropy' , random_state=0)
lr = lr.fit(X_train,Y_train)

###Accuracy

In [ ]:
YA=Y_test
Yp=lr.predict(X_test)
from sklearn.metrics import accuracy_score
acc=accuracy_score(YA,Yp)
acc

0.8333333333333334

##Multinomial Naive Bayes

In [ ]:
lr = MultinomialNB(alpha=1.0,fit_prior=True)
lr = lr.fit(X_train,Y_train)

###Accuracy

In [ ]:
YA=Y_test
Yp=lr.predict(X_test)
from sklearn.metrics import accuracy_score
acc=accuracy_score(YA,Yp)
acc

0.875

##SVM

In [ ]:
lr = SVC()
lr = lr.fit(X_train,Y_train)

###Accuracy

In [ ]:
YA=Y_test
Yp=lr.predict(X_test)
from sklearn.metrics import accuracy_score
acc=accuracy_score(YA,Yp)
acc

0.8333333333333334

###Accuracy

##Decision Tree Classifier

In [ ]:
lr = DecisionTreeClassifier()
lr = lr.fit(X_train,Y_train)

###Accuracy

In [ ]:
YA=Y_test
Yp=lr.predict(X_test)
from sklearn.metrics import accuracy_score
acc=accuracy_score(YA,Yp)
acc

0.625

In [ ]:
# YA=Y_test
# Yp=lr.predict(X_test)

In [ ]:
# from sklearn.metrics import accuracy_score
# acc=accuracy_score(YA,Yp)
# acc

In [ ]:
# text=["Raindrops and thunderstorms in #Delhi may impact our flight operations. Please keep enough travel time in hand to avoid any hassle. "]
# text_vec=my_tfidf.transform(text).toarray()
# text_vec.shape
# lr.predict(text_vec)

##Saving Model and Tfidf

In [ ]:
# model_name = "trained_model.sav"
# vec_file = "vectorizer.pickle"
# pickle.dump(lr,open(model_name,'wb'))
# pickle.dump(my_tfidf,open(vec_file,'wb'))

In [ ]:
#Pickle File test

In [ ]:
# model = pickle.load(open(model_name,'rb'))
# tfidf = pickle.load(open(vec_file,'rb'))

# text=["Raindrops and thunderstorms in #Delhi may impact our flight operations. Please keep enough travel time in hand to avoid any hassle. "]
# text = tfidf.transform(text).toarray()
# print(model.predict(text))